In [ ]:
import tensorflow as tf
import numpy as np
import math
%matplotlib inline
from __future__ import division, print_function, absolute_import
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import os

from sklearn.cross_validation import train_test_split


def lrelu(x, leak=0.2, name="lrelu"):
    """Leaky rectifier.
    Parameters
    ----------
    x : Tensor
        The tensor to apply the nonlinearity to.
    leak : float, optional
        Leakage parameter.
    name : str, optional
        Variable scope to use.
    Returns
    -------
    x : Tensor
        Output of the nonlinearity.
    """
    with tf.variable_scope(name):
        f1 = 0.5 * (1 + leak)
        f2 = 0.5 * (1 - leak)
        return f1 * x + f2 * abs(x)

# %%
def autoencoder(input_shape=[None, 784],
                n_filters=[1, 10, 10, 10],
                filter_sizes=[3, 3, 3],
                corruption=False):
    """Build a deep denoising autoencoder w/ tied weights.
    Parameters
    ----------
    input_shape : list, optional
        Description
    n_filters : list, optional
        Description
    filter_sizes : list, optional
        Description
    Returns
    -------
    x : Tensor
        Input placeholder to the network
    z : Tensor
        Inner-most latent representation
    y : Tensor
        Output reconstruction of the input
    cost : Tensor
        Overall cost to use for training
    Raises
    ------
    ValueError
        Description
    """
    # %%
    # input to the network
    x = tf.placeholder(
        tf.float32, input_shape, name='x')


    # %%
    # ensure 2-d is converted to square tensor.
    if len(x.get_shape()) == 2:
        x_dim = np.sqrt(x.get_shape().as_list()[1])
        if x_dim != int(x_dim):
            raise ValueError('Unsupported input dimensions')
        x_dim = int(x_dim)
        x_tensor = tf.reshape(
            x, [-1, x_dim, x_dim, n_filters[0]])
    elif len(x.get_shape()) == 4:
        x_tensor = x
    else:
        raise ValueError('Unsupported input dimensions')
    current_input = x_tensor

    # %%
    # Optionally apply denoising autoencoder
    if corruption:
        current_input = corrupt(current_input)

    # %%
    # Build the encoder
    encoder_weights = []
    encoder_ops = []
    shapes = []
    for layer_i, n_output in enumerate(n_filters[1:]):
        n_input = current_input.get_shape().as_list()[3]
        shapes.append(current_input.get_shape().as_list())
        W = tf.Variable(
            tf.random_uniform([
                filter_sizes[layer_i],
                filter_sizes[layer_i],
                n_input, n_output],
                -1.0 / math.sqrt(n_input),
                1.0 / math.sqrt(n_input)))
        b = tf.Variable(tf.zeros([n_output]))
        encoder_weights.append(W)
        output = lrelu(
            tf.add(tf.nn.conv2d(
                current_input, W, strides=[1, 1, 1, 1], padding='SAME'), b))
        encoder_ops.append(output)
        current_input = output

    # %%
    # store the latent representation
    z = current_input
    encoder_weights.reverse()
    shapes.reverse()

    # %%
    # Build the decoder using the same weights
    for layer_i, shape in enumerate(shapes):
        W = encoder_weights[layer_i]
        b = tf.Variable(tf.zeros([W.get_shape().as_list()[2]]))
        output = lrelu(tf.add(
            tf.nn.conv2d_transpose(
                current_input, W,
                tf.pack([tf.shape(x)[0], shape[1], shape[2], shape[3]]),
                strides=[1, 1, 1, 1], padding='SAME'), b))
        current_input = output

    # %%
    # now have the reconstruction through the network
    y = current_input
    # cost function measures pixel-wise difference
    cost = tf.reduce_sum(tf.square(y - x_tensor))

    # %%
    return {'x': x, 'z': z, 'y': y, 'cost': cost, "encoder": encoder_ops}


# %%
def test_mnist():
    """Test the convolutional autoencder using MNIST."""
    # %%
    import tensorflow as tf
    import tensorflow.examples.tutorials.mnist.input_data as input_data
    import matplotlib.pyplot as plt

    # %%
    # load MNIST as before
    mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
    mean_img = np.mean(mnist.train.images, axis=0)
    ae = autoencoder()

    # %%
    learning_rate = 0.01
    optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(ae['cost'])

    # %%
    # We create a session to use the graph
    sess = tf.Session()
    sess.run(tf.initialize_all_variables())

    # %%
    # Fit all training data
    batch_size = 100
    n_epochs = 50
    for epoch_i in range(n_epochs):
        for batch_i in range(mnist.train.num_examples // batch_size):
            batch_xs, _ = mnist.train.next_batch(batch_size)
            train = np.array([img - mean_img for img in batch_xs])
            sess.run(optimizer, feed_dict={ae['x']: train})
        print(epoch_i, sess.run(ae['cost'], feed_dict={ae['x']: train}))

    # %%
    # Plot example reconstructions
    n_examples = 10
    test_xs, _ = mnist.test.next_batch(n_examples)
    test_xs_norm = np.array([img - mean_img for img in test_xs])
    recon = sess.run(ae['y'], feed_dict={ae['x']: test_xs_norm})
    print(recon.shape)
    fig, axs = plt.subplots(2, n_examples, figsize=(10, 2))
    for example_i in range(n_examples):
        axs[0][example_i].imshow(
            np.reshape(test_xs[example_i, :], (28, 28)), cmap="Greys_r")
        axs[1][example_i].imshow(
            np.reshape(
                np.reshape(recon[example_i, ...], (784,)) + mean_img,
                (28, 28)), cmap="Greys_r")
    fig.show()
    plt.draw()
    
    ae["session"] = sess
    
    return ae

In [ ]:
ae = test_mnist()

In [ ]:
print(ae["encoder"])
print(ae["session"])

In [ ]:
sess = ae["session"]

In [ ]:
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
mnist.train.next_batch(10)[0].shape
mean_img = np.mean(mnist.train.images, axis=0)

In [ ]:
batch_xs, batch_ys = mnist.train.next_batch(10000)
train = np.array([img - mean_img for img in batch_xs])
layers = [sess.run(ae["encoder"][i], feed_dict={ae['x']: train}) for i in range(len(ae["encoder"]))]

In [ ]:
print(layers[0].shape)
print(layers[1].shape)
print(layers[2].shape)

In [ ]:
len(layers)

In [ ]:
ravels = (np.array([row.ravel() for row in layers[i]]) for i in range(len(ae["encoder"])))
# ravel_0 = np.array([row.ravel() for row in layers[0]])
# ravel_1 = np.array([row.ravel() for row in layers[1]])
# ravel_2 = np.array([row.ravel() for row in layers[2]])
# ravel_3 = np.array([row.ravel() for row in layers[3]])

In [ ]:
combined = np.hstack(ravels)

In [ ]:
def cluster_acc(Y_pred, Y):
    """
    Finds the cluster accuracy
    """
    from sklearn.utils.linear_assignment_ import linear_assignment
    assert Y_pred.size == Y.size
    D = max(Y_pred.max(), Y.max())+1
    w = np.zeros((D,D), dtype=np.int64)
    for i in xrange(Y_pred.size):
        w[Y_pred[i], Y[i]] += 1
    ind = linear_assignment(w.max() - w)
    return sum([w[i,j] for i,j in ind])*1.0/Y_pred.size, w

In [ ]:
# Scale and visualize the embedding vectors
def plot_embedding(X, y, title=None):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    plt.figure(figsize=(20, 10))
    ax = plt.subplot(111)
    for i in range(X.shape[0]):
        plt.text(X[i, 0], X[i, 1], str(y[i]),
                 color=plt.cm.Set1(y[i] / 10.),
                 fontdict={'weight': 'bold', 'size': 12})

    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)

vectorized_imgs = combined
y = [np.argmax(row) for row in batch_ys]

from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=0)
np.set_printoptions(suppress=True)
X_tsne = tsne.fit_transform(vectorized_imgs)

In [ ]:
plot_embedding(X_tsne, y)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_tsne, y)

In [ ]:
knn.score(X_tsne, y)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot_embedding_3d(X, y, title=None):
    fig = plt.figure(figsize=(20, 10))
    ax = fig.add_subplot(111, projection='3d')    
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    for i in range(X.shape[0]):
        ax.text(X[i, 0], X[i, 1], X[i, 2], str(y[i]),
                 color=plt.cm.Set1(y[i] / 10.),
                 fontdict={'size': 5})
    ax.set_zticks([])
    ax.set_xticks([]), ax.set_yticks([])

    if title is not None:
        plt.title(title)    

In [ ]:
tsne = TSNE(n_components=3, random_state=0)
np.set_printoptions(suppress=True)
X_tsne_3 = tsne.fit_transform(vectorized_imgs)

In [ ]:
plot_embedding_3d(X_tsne_3, y)

# Let's look at the features

In [ ]:
layers[0].shape

In [ ]:
y = np.array(y)
by_classes_index = {i: np.where(y==i)[0] for i in range(10)}

In [ ]:
by_classes_index[1][0]

In [ ]:
classes = [0, 1, 3, 5, 7] # max is 10
num_examples = 10
layer_index = 2 # 0, 1, 2, 3
filter_index = 3 # 0-9
fig, axs = plt.subplots(len(classes), num_examples, figsize=(10, 5))
for idx, class_i in enumerate(classes):
    for example_i in range(num_examples):
        axs[idx][example_i].imshow(layers[layer_index][by_classes_index[class_i][example_i], :, :, filter_index])
        axs[idx][example_i].axis("off")


In [ ]:
ae["encoder"][0].eval(session=sess, feed_dict={ae['x']: train}).shape

# What's the distance between pairs of classes?

In [ ]:
flattened = [np.array([row.ravel() for row in layers[i]]) for i in range(len(ae["encoder"]))]

In [ ]:
flattened[0].shape

In [ ]:
from scipy.spatial import distance_matrix

zeros = flattened[0][by_classes_index[0]]
ones = flattened[0][by_classes_index[1]]

In [ ]:
print(zeros.shape)
print(ones.shape)

In [ ]:
import time
t1 = time.time()
time.sleep(1)
t2 = time.time()
print(t2 - t1)

In [ ]:
t1 = time.time()
arr = distance_matrix(zeros, ones)
t2 = time.time()
print(t2-t1)

In [ ]:
np.mean(arr)

In [ ]:
arr = distance_matrix(zeros, zeros)
np.mean(arr)

In [ ]:
avg_distance_matrix = np.zeros((10, 10))

for row_i in range(avg_distance_matrix.shape[0]):
    for col_i in range(avg_distance_matrix.shape[1]):
        if avg_distance_matrix[col_i][row_i] == 0:
            left = flattened[0][by_classes_index[row_i]]
            right = flattened[0][by_classes_index[col_i]]
            arr = distance_matrix(left, right)
            m = np.mean(arr)
            avg_distance_matrix[row_i][col_i] = m
            avg_distance_matrix[col_i][row_i] = m

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

a = plt.imshow(avg_distance_matrix, cmap='OrRd', interpolation='nearest')
cbar = plt.colorbar(a)
# cbar.ax.set_yticklabels([np.minimum(avg_distance_matrix), np.maximum(avg_distance_matrix)])
plt.show()

In [ ]:
avg_distance_matrix = np.zeros((10, 10))

for row_i in range(avg_distance_matrix.shape[0]):
    for col_i in range(avg_distance_matrix.shape[1]):
        if avg_distance_matrix[col_i][row_i] == 0:
            left = flattened[1][by_classes_index[row_i]]
            right = flattened[1][by_classes_index[col_i]]
            arr = distance_matrix(left, right)
            m = np.mean(arr)
            avg_distance_matrix[row_i][col_i] = m
            avg_distance_matrix[col_i][row_i] = m

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

a = plt.imshow(avg_distance_matrix, cmap='OrRd', interpolation='nearest')
cbar = plt.colorbar(a)
# cbar.ax.set_yticklabels([np.minimum(avg_distance_matrix), np.maximum(avg_distance_matrix)])
plt.show()

In [ ]:
combined.shape

In [ ]:
avg_distance_matrix = np.zeros((10, 10))

for row_i in range(avg_distance_matrix.shape[0]):
    for col_i in range(avg_distance_matrix.shape[1]):
        if avg_distance_matrix[col_i][row_i] == 0:
            left = combined[by_classes_index[row_i]]
            right = combined[by_classes_index[col_i]]
            arr = distance_matrix(left, right)
            m = np.mean(arr)
            avg_distance_matrix[row_i][col_i] = m
            avg_distance_matrix[col_i][row_i] = m

a = plt.imshow(avg_distance_matrix, cmap='OrRd', interpolation='nearest')
cbar = plt.colorbar(a)
plt.show()

In [ ]:
avg_distance_matrix = np.zeros((10, 10))

for row_i in range(avg_distance_matrix.shape[0]):
    for col_i in range(avg_distance_matrix.shape[1]):
        if avg_distance_matrix[col_i][row_i] == 0:
            left = batch_xs[by_classes_index[row_i]]
            right = batch_xs[by_classes_index[col_i]]
            arr = distance_matrix(left, right)
            m = np.mean(arr)
            avg_distance_matrix[row_i][col_i] = m
            avg_distance_matrix[col_i][row_i] = m

a = plt.imshow(avg_distance_matrix, cmap='OrRd', interpolation='nearest')
cbar = plt.colorbar(a)
plt.show()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_tsne_3, y)

In [ ]:
knn.score(X_tsne_3, y)

In [ ]:
tsne = TSNE(n_components=4, random_state=0)
np.set_printoptions(suppress=True)
X_tsne_4 = tsne.fit_transform(vectorized_imgs)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_tsne_4, y)

In [ ]:
knn.score(X_tsne_4, y)

In [ ]:
plot_embedding_3d(X_tsne_4[:, 1:], y)

In [ ]:
plot_embedding(X_tsne_4[:, 1:], y)

In [ ]:
plot_embedding(X_tsne_4[:, 2:], y)

In [ ]:
tsne = TSNE(n_components=5, random_state=0)
np.set_printoptions(suppress=True)
X_tsne_5 = tsne.fit_transform(vectorized_imgs)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_tsne_5, y)

In [ ]:
knn.score(X_tsne_5, y)

In [ ]:
combined.shape

In [ ]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_tsne_5, y)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(combined, y)

In [ ]:
knn.score(combined, y)

In [ ]:
mnist.train

# Cluster with only t-SNE

In [ ]:
batch_xs, batch_ys = mnist.train.next_batch(10000)
print(batch_xs.shape)
print(batch_ys.shape)

In [ ]:
import numpy as np
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2)
np.set_printoptions(suppress=True)
X_tsne = tsne.fit_transform(batch_xs)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_tsne, batch_ys)

In [ ]:
knn.score(X_tsne, batch_ys)

In [ ]:
y = [np.argmax(row) for row in batch_ys]

In [ ]:
def plot_embedding(X, y, title=None):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    plt.figure(figsize=(20, 10))
    ax = plt.subplot(111)
    for i in range(X.shape[0]):
        plt.text(X[i, 0], X[i, 1], str(y[i]),
                 color=plt.cm.Set1(y[i] / 10.),
                 fontdict={'weight': 'bold', 'size': 12})

    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)
        
plot_embedding(X_tsne, y)

# Clustering with just k-NN

In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
mnist.train.next_batch(10)[0].shape
mean_img = np.mean(mnist.train.images, axis=0)
batch_xs, batch_ys = mnist.train.next_batch(10000)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(batch_xs, batch_ys)

In [ ]:
knn.score(batch_xs, batch_ys)